In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import expit
from scipy.optimize import curve_fit


In [43]:
def step_sim(step_size = 8.0, SNR = 0.5, total_time_point=401):
    np.random.seed(20181118)
    
    steps = np.piecewise(np.arange(1, total_time_point), 
                            [
                                np.arange(1, total_time_point) < 100, 

                                np.logical_and(np.arange(1, total_time_point) >= 100, np.arange(1, total_time_point) < 200), 

                                np.logical_and(np.arange(1, total_time_point) >= 200, np.arange(1, total_time_point) < 250), 
                                
                                np.logical_and(np.arange(1, total_time_point) >= 250, np.arange(1, total_time_point) < 350), 
                                
                                np.logical_and(np.arange(1, total_time_point) >= 350, np.arange(1, total_time_point) < 500), 

                                np.arange(1, total_time_point) >= 500
                            ], 
                            [
                                lambda x: 300, 
                                lambda x: 100, 
                                lambda x: 50, 
                                lambda x: 350,
                                lambda x: 20,
                                lambda x: 350,
                            ])
    x = np.arange(1, total_time_point)
    noise_STD = step_size / SNR
    noise = np.random.normal(scale=noise_STD, size=len(x))
    # y = true_fct + np.random.standard_t(1, size=len(x))
    y = steps + noise
    return x, y, steps



x_data, y_data , steps = step_sim(total_time_point=601)
plt.plot(x_data, y_data, '-b', label="noise")
plt.plot(x_data, steps, '-r', linewidth=1, label="raw")

In [42]:
from scipy.optimize import curve_fit

def get_step_location_size(x, y, adust_location=0):
    # Heaviside step function
    # def step_fun(x,a,b,c): return a * (np.sign(x-b) + c)
    # p0=[np.max(y), np.median(x), np.min(y)]

    # Logistic function
    def step_fun(x, step_size ,location, k, b):
        y = step_size / (1 + np.exp(-k*(x-location))) + b
        return (y)

    # p0 = [ np.max(y_data),1,np.min(y_data)]
    p0=[np.max(y), np.median(x),1, np.min(y)]


    popt, pcov, infodict, mesg, ier = curve_fit(step_fun, x, y,p0=p0, full_output=True, maxfev = 10000)
    fitted_data = step_fun(x, *popt)

    # popt : [-97.86263752  99.1317172  -18.4030292  195.17399986]
    # value was going from 100 to 200
    step_size , location, k, b = popt
    return location + adust_location, step_size, (k,b)
get_step_location_size(x_data, y_data)

C:\Users\CellphyLab\AppData\Local\Temp\ipykernel_21512\130051023.py:10: RuntimeWarning: overflow encountered in exp
  y = L / (1 + np.exp(-k*(x-x0))) + b


(349.63737776470424,
 -67.7467421796568,
 (76.15523012908508, 220.76202117354717))

In [ ]:
def step_generator(x, y):
    step_adjust = 0
    location, step_size = get_step_location_size(x, y)
    left_section = y[:int(location)-1]
    right_section = y[int(location):]
    next_section = None
    if (abs(step_size) * np.sqrt(len(left_section)) ) > (abs(step_size) * np.sqrt(len(right_section)) ):
        next_section = left_section
    else: 
        next_section = right_section



In [39]:
# step_generator(x_data, y_data)
y_data[348] + (-67)

292.8390728211739

In [41]:
y_data[98] + (-97.86)

207.25406023852833